# Sequence Labeling in python

### Initialise the transition, start and emission matrix . The states stand for high and low.  The HMM model is given in the assignment itself. 

In [5]:
import numpy as np

P= np.array([[0.6, 0.4],[0.5,0.5]])

S= np.array([0.5, 0.5])

O= np.array([[0.3,0.2,0.2,0.3],[0.2,0.3,0.3,0.2]])

state={}
state[0]='L'
state[1]='H'

DNA={}
DNA['A']=0
DNA['C']=1
DNA['G']=2
DNA['T']=3


### A stupid attempt to show you why the exhaustive search is a bad, bad option for HMM modelling. 

In [6]:
from itertools import product

import time 
def exhaustive_search(sequence):
    
    M= len(sequence)
    state_len= len(S)

    # track the best sequence and its score
    best=(None,float('-inf'))
    
    # basically loop will run for |states|^M 
    for ss in product(range(state_len),repeat=M):
        #print(ss)
        score= S[ss[0]]*O[ss[0],DNA[sequence[0]]]
        
        for i in range(1,M):
            score*= P[ss[i-1],ss[i]]*O[ss[i],DNA[sequence[i]]]
    
        if score > best[1]:
            best= (ss,score)
    
    return best


In [7]:
sequences=['GGC','GGCAAGATCAT','GAGAGGAGAGAGAGAGAGA']

import time
for sequence in sequences:
    
    t=time.time()
    best=exhaustive_search(sequence)
    t2=time.time()-t
    
    print('For the sequence '+ sequence+ ' of length '+ str(len(sequence))+' time taken was '+ str(round(t2,3))+'s' )
    print('The sequence '+ ','.join([state[k] for k in best[0]])+ ' gave the best score of '+ str(best[1]))
    print('\n')

For the sequence GGC of length 3 time taken was 0.0s
The sequence H,H,H gave the best score of 0.003375


For the sequence GGCAAGATCAT of length 11 time taken was 0.03s
The sequence H,H,H,L,L,L,L,L,L,L,L gave the best score of 1.3774950719999997e-09


For the sequence GAGAGGAGAGAGAGAGAGA of length 19 time taken was 11.367s
The sequence H,L,L,L,H,H,L,L,L,L,L,L,L,L,L,L,L,L,L gave the best score of 1.3326697514029538e-16




# Dataset: Brown corpus tagged with the Universal Tagset.

## This will be your training set. The remaining 100 sentences will be used as your test data.

In [23]:
import nltk
nltk.download('universal_tagset')

from nltk.corpus import treebank,brown

corpus = brown.tagged_sents(tagset='universal')[:-100] 
#print(corpus[0])
tag_dict={}
word_dict={}

for sent in corpus:
    for elem in sent:
        w = elem[0]
        tag= elem[1]
        if w not in word_dict:
            word_dict[w]=0
        if tag not in tag_dict:
            tag_dict[tag]=0
        word_dict[w]+=1
        tag_dict[tag]+=1

print('Number of words(M): ',len(word_dict))
print('Number of tags(N): ',len(tag_dict))
print(tag_dict)
        
test_data= brown.tagged_sents(tagset='universal')[-10:]


[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Vasu\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


Number of words(M):  55907
Number of tags(N):  12
{'DET': 136724, 'NOUN': 275075, 'ADJ': 83581, 'VERB': 182380, 'ADP': 144483, '.': 147231, 'ADV': 56115, 'CONJ': 38067, 'PRT': 29759, 'PRON': 49174, 'NUM': 14853, 'X': 1369}


# Hidden Markov Model 

## Learn transition,start and emission matrices from training data

In [10]:
brown_tags_words = [ ]
for sent in corpus:  
    brown_tags_words.append( ("START", "START") )
    brown_tags_words.extend([ (tag, word) for (word, tag) in sent ])
    brown_tags_words.append( ("END", "END") )
    
print(brown_tags_words[0:30])

[('START', 'START'), ('DET', 'The'), ('NOUN', 'Fulton'), ('NOUN', 'County'), ('ADJ', 'Grand'), ('NOUN', 'Jury'), ('VERB', 'said'), ('NOUN', 'Friday'), ('DET', 'an'), ('NOUN', 'investigation'), ('ADP', 'of'), ('NOUN', "Atlanta's"), ('ADJ', 'recent'), ('NOUN', 'primary'), ('NOUN', 'election'), ('VERB', 'produced'), ('.', '``'), ('DET', 'no'), ('NOUN', 'evidence'), ('.', "''"), ('ADP', 'that'), ('DET', 'any'), ('NOUN', 'irregularities'), ('VERB', 'took'), ('NOUN', 'place'), ('.', '.'), ('END', 'END'), ('START', 'START'), ('DET', 'The'), ('NOUN', 'jury')]


In [11]:
import nltk

# conditional frequency distribution of the word given the tags
CFD_tag_words = nltk.ConditionalFreqDist(brown_tags_words)
# conditional probability distribution of the word given the tags
# P(wi | ti)
CPD_tag_words = nltk.ConditionalProbDist(CFD_tag_words, nltk.MLEProbDist)

brown_tags = [tag for (tag, word) in brown_tags_words ]

# make conditional frequency distribution: count(t{i-1} ti)
CFD_tags= nltk.ConditionalFreqDist(nltk.bigrams(brown_tags))
# make conditional probability distribution, using maximum likelihood estimate:
# P(ti | t{i-1})
CPD_tags = nltk.ConditionalProbDist(CFD_tags, nltk.MLEProbDist)

tag_trans_matrix=[]
j=-1
for tag1 in tag_dict:
    tag_trans_matrix.append([])
    j=j+1
    for tag2 in tag_dict:
        tag_trans_matrix[j].append(CPD_tags[tag1].prob(tag2))
print("State Transition matrix:")
print(tag_trans_matrix)

State Transition matrix:
[[0.005917029928907872, 0.6263860039203066, 0.23980427723004008, 0.06470700096544864, 0.009091308036628536, 0.012748310464878149, 0.017502413621602646, 0.0006436324273719318, 0.002011351335537287, 0.009895848570843451, 0.009764196483426465, 0.0013969749275913519], [0.01550122693810779, 0.14934835953830775, 0.012894665091338726, 0.1589093883486322, 0.24437698809415614, 0.2835844769608289, 0.026370989730073617, 0.0596782695628465, 0.01782786512769245, 0.01978369535581205, 0.008077796964464238, 0.0003308188675815687], [0.005850611981191898, 0.65281583134923, 0.05690288462688889, 0.01746808485182039, 0.08844115289360022, 0.10033380792285328, 0.009643339993539201, 0.037604240198131154, 0.019286679987078403, 0.003804692454026633, 0.006987233940728156, 0.0004905421088524905], [0.16292904923785503, 0.0975710055927185, 0.05753920386007238, 0.1843020067989911, 0.1691797346200241, 0.08060642614321746, 0.10326241912490404, 0.014376576378988924, 0.06559381511130606, 0.05492

## Viterbi Algorithm to get the best POS tag

In [12]:

def Viterbi(sentence):
    sentlen = len(sentence)
    distinct_tags = set(brown_tags)

    viterbi = [ ]
    backpointer = [ ]

    viterbi_init = { }
    backpointer_init = { }

    for tag in distinct_tags:
        if tag == "START": continue
        viterbi_init[ tag ] = CPD_tags["START"].prob(tag) * CPD_tag_words[tag].prob(sentence[0])
        backpointer_init[ tag ] = "START"
    viterbi.append(viterbi_init)
    backpointer.append(backpointer_init)

    for wordindex in range(1, len(sentence)):
        cur_viterbi = { }
        cur_backpointer = { }
        prev_viterbi = viterbi[-1]
            
        for tag in distinct_tags:
            if tag == "START": continue
            if sentence[wordindex] not in word_dict.keys():
                best_prevtag = max(prev_viterbi.keys(),key = lambda prevtag: \
                    prev_viterbi[ prevtag ] * CPD_tags[prevtag].prob(tag) *0.0001)
                cur_viterbi[tag] = prev_viterbi[ best_prevtag ] *\
                                    CPD_tags[best_prevtag].prob(tag) * 0.0001
            else:
                best_prevtag = max(prev_viterbi.keys(),key = lambda prevtag: \
                    prev_viterbi[ prevtag ] * CPD_tags[prevtag].prob(tag) * 
                                    CPD_tag_words[tag].prob(sentence[wordindex]))
                cur_viterbi[tag] = prev_viterbi[ best_prevtag ] *\
                                    CPD_tags[best_prevtag].prob(tag) *\
                                    CPD_tag_words[tag].prob(sentence[wordindex])
            cur_backpointer[tag] = best_prevtag

        viterbi.append(cur_viterbi)
        backpointer.append(cur_backpointer)

    prev_viterbi = viterbi[-1]
    best_prevtag = max(prev_viterbi.keys(),key = lambda prevtag: prev_viterbi[ prevtag ] *\
                       CPD_tags[prevtag].prob("END"))

    prob_best_seq = prev_viterbi[ best_prevtag ] * CPD_tags[ best_prevtag].prob("END")
    best_tag_seq = [ "END", best_prevtag ]

    backpointer.reverse()
    current_best_tag = best_prevtag
    for bp in backpointer:
        best_tag_seq.append(bp[current_best_tag])
        current_best_tag = bp[current_best_tag]

    best_tag_seq.reverse()
    
    return best_tag_seq[1:-1]
    
#sentence = ['I', "can't", 'drive', 'a', 'car', '.']
sentence = ['you', "can't", 'very', 'well', 'sidle', 'up', 'to', 'people', 'on', 'the', 'street', 'and', 'ask', 'if', 'they', 'want', 'to', 'buy', 'a', 'hot', 'Bodhisattva', '.']

tag_seq=Viterbi(sentence)
print("\nThe given sentence:",sentence)
print("\nThe POS tags:",tag_seq)


The given sentence: ['you', "can't", 'very', 'well', 'sidle', 'up', 'to', 'people', 'on', 'the', 'street', 'and', 'ask', 'if', 'they', 'want', 'to', 'buy', 'a', 'hot', 'Bodhisattva', '.']

The POS tags: ['PRON', 'VERB', 'ADV', 'ADV', 'VERB', 'PRT', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', 'CONJ', 'VERB', 'ADP', 'PRON', 'VERB', 'PRT', 'VERB', 'DET', 'ADJ', 'NOUN', '.']


## Results with HMM using Viterbi

In [13]:
from sklearn.metrics import accuracy_score

test_list=[]
y_true=[]
y_pred=[]
i=-1
for sent in test_data:
    test_list.append([])
    i=i+1
    for elem in sent:
        test_list[i].append(elem[0])
        y_true.append(elem[1])
        
print(test_list[0])
        
print("\nActual tags:")
print("************")
print(y_true)

for sent in test_list:
    y_pred=y_pred+Viterbi(sent)
    
print("\nPredicted tags:")
print("************")
print(y_pred)
print("\nTotal testing accuracy: ",accuracy_score(y_true, y_pred))


['you', "can't", 'very', 'well', 'sidle', 'up', 'to', 'people', 'on', 'the', 'street', 'and', 'ask', 'if', 'they', 'want', 'to', 'buy', 'a', 'hot', 'Bodhisattva', '.']

Actual tags:
************
['PRON', 'VERB', 'ADV', 'ADV', 'VERB', 'ADP', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', 'CONJ', 'VERB', 'ADP', 'PRON', 'VERB', 'PRT', 'VERB', 'DET', 'ADJ', 'NOUN', '.', 'ADV', '.', 'ADP', 'PRT', 'VERB', 'PRT', 'VERB', 'X', 'X', 'X', 'ADV', 'ADV', 'ADP', 'NOUN', '.', 'NOUN', '.', 'NOUN', 'NOUN', '.', 'DET', 'NOUN', 'NOUN', '.', 'NOUN', 'NOUN', '.', 'NOUN', '.', 'CONJ', 'DET', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'PRT', 'VERB', '.', 'PRON', 'VERB', 'VERB', 'PRT', 'VERB', 'VERB', 'ADV', '.', 'DET', 'NOUN', '.', 'ADP', 'PRON', 'VERB', 'ADJ', 'ADV', 'PRT', 'VERB', 'DET', 'NOUN', '.', 'VERB', 'ADP', 'DET', 'ADJ', 'NOUN', 'PRT', 'VERB', 'ADP', 'DET', 'ADJ', '.', 'VERB', 'PRON', 'PRT', '.', 'PRT', 'NOUN', 'CONJ', 'DET', 'NOUN', '.', 'ADP', 'PRON', 'VERB', 'VERB', 'ADP', 'PRON', 'PRT', 'VERB',

# Module to implement CRF 

## Feature definition using each word

In [17]:
!pip3 install sklearn-crfsuite

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

train_sents= corpus

def word2features(sent,i):
    word = sent[i][0]
    features ={
    'bias': 1.0,
    'word':word,
    'is_first': i == 0,
    'is_last': i == len(sent) - 1,
    'is_capitalized': word[0].upper() == word[0],
    'is_all_caps': word.upper() == word,
    'is_all_lower': word.lower() == word,
    'prefix-1': word[0],
    'prefix-2': word[:2],
    'prefix-3': word[:3],
    'suffix-1': word[-1],
    'suffix-2': word[-2:],
    'suffix-3': word[-3:],
    'prev_word': '' if i == 0 else sent[i - 1][0],
    'next_word': '' if i == len(sent) - 1 else sent[i + 1][0],
    'has_hyphen': '-' in word,
    'is_numeric': word.isdigit(),
    'capitals_inside': word[1:].lower() != word[1:]
    }
                
    return features

def sent2features(sent):
    return [word2features(sent,i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for i,label in sent]

#print(sent2features(corpus[0]))


In [18]:
X_train=[sent2features(s) for s in train_sents]
y_train=[sent2labels(s) for s in train_sents]

X_test=[sent2features(s) for s in test_data]
y_test=[sent2labels(s) for s in test_data]



## Implementation of CRF using sklearn_crfsuite

In [20]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

## Results with CRF

In [21]:
y_pred = crf.predict(X_test)
labels=list(crf.classes_)

metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

C:\Users\Vasu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1492: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.9749640807442314

In [22]:
sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           .      1.000     1.000     1.000        33
           X      1.000     1.000     1.000         3
         ADJ      1.000     0.944     0.971        18
         ADP      0.962     0.926     0.943        27
         ADV      0.900     1.000     0.947         9
        VERB      0.972     1.000     0.986        35
         DET      0.971     1.000     0.985        33
        CONJ      1.000     0.857     0.923         7
        NOUN      1.000     0.980     0.990        51
        PRON      1.000     0.917     0.957        12
         PRT      0.846     1.000     0.917        11
         NUM      0.000     0.000     0.000         0

   micro avg      0.975     0.975     0.975       239
   macro avg      0.888     0.885     0.885       239
weighted avg      0.977     0.975     0.975       239



C:\Users\Vasu\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['.', 'X', 'ADJ', 'ADP', 'ADV', 'VERB', 'DET', 'CONJ', 'NOUN', 'PRON', 'PRT', 'NUM'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
C:\Users\Vasu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vasu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vasu